# Aanalysis code in year 4

In [ ]:
import csv
import sqlite3

In [ ]:
indexpath = 'dbv3-index.db'
corepath = 'dbv3-core.db'
servicepath = 'dbv3-service.db'
utilpath = 'dbv3-util.db'

In [ ]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute(f'''ATTACH DATABASE '{indexpath}' AS DBINDEX;''')
cur.execute(f'''ATTACH DATABASE '{corepath}' AS DBCORE;''')
cur.execute(f'''ATTACH DATABASE '{servicepath}' AS DBSERVICE;''')
cur.execute(f'''ATTACH DATABASE '{utilpath}' AS DBUTIL;''')
conn.commit()

In [ ]:
tagpath = 'datasets/NamedCluster-Year4.csv'

In [ ]:
with open(tagpath, 'r') as f:
    reader = csv.DictReader(f)
    print(reader.fieldnames)